# Dev Notebook

This notebook loads the version of trulens_eval from the enclosing repo folder. You can use this to debug or devlop trulens_eval features.

In [ ]:
# pip uninstall -y trulens
# pip install git+https://github.com/truera/trulens@piotrm/azure_bugfixes#subdirectory=trulens

# trulens_eval notebook dev

# %load_ext autoreload
# %autoreload 2
from pathlib import Path
import sys

base = Path().cwd()
while not (base / "trulens_eval").exists():
    base = base.parent


# import os
# if os.path.exists("default.sqlite"):
#    os.unlink("default.sqlite")

print(base)

# import shutil
# shutil.copy(base / "release_dbs" / "0.19.0" / "default.sqlite", "default.sqlite")

# If running from github repo, can use this:
sys.path.append(str(base))

# Uncomment for more debugging printouts.
"""
import logging
root = logging.getLogger()
root.setLevel(logging.DEBUG)

handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)
root.addHandler(handler)
"""

from trulens.core.utils.keys import check_keys

check_keys("OPENAI_API_KEY", "HUGGINGFACE_API_KEY")

from trulens.core import Tru
from trulens.dashboard import run_dashboard

tru = Tru(database_prefix="dev")
# tru.reset_database()
# run_dashboard(tru=tru, _dev=base, force=True)
# tru.db.migrate_database()

In [ ]:
# tru.db.migrate_database()
tru.migrate_database()

In [ ]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents import create_structured_chat_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_core.messages import AIMessage
from langchain_core.messages import HumanMessage

prompt = hub.pull("hwchase17/structured-chat-agent")
model = ChatOpenAI()
tools = []

agent = create_structured_chat_agent(model, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

In [ ]:
res1 = agent_executor.invoke({"input": "hi"})

# Using with chat history

res2 = agent_executor.invoke(
    {
        "input": "what's my name?",
        "chat_history": [
            HumanMessage(content="hi! my name is bob"),
            AIMessage(content="Hello Bob! How can I assist you today?"),
        ],
    }
)

In [ ]:
res2

In [ ]:
from trulens.instrument.langchain import TruChain

recorder = TruChain(agent_executor)

In [ ]:
with recorder as recs:
    agent_executor.invoke({"input": "hi"})

    # Using with chat history

    """
    agent_executor.invoke(
        {
            "input": "what's my name?",
            "chat_history": [
                HumanMessage(content="hi! my name is bob"),
                AIMessage(content="Hello Bob! How can I assist you today?"),
            ],
        }
    )
    """

In [ ]:
record = recs.get()

In [ ]:
record.main_output

In [ ]:
for t in tru.db.orm.registry.values():
    print(t)

In [ ]:
from trulens.database.utils import copy_database

In [ ]:
tru.db

In [ ]:
copy_database(
    "sqlite:///default.sqlite",
    "sqlite:///default2.sqlite",
    src_prefix="dev",
    tgt_prefix="dev",
)

In [ ]:
from trulens.instrument.llamaindex import TruLlama

check_keys("OPENAI_API_KEY", "HUGGINGFACE_API_KEY")
import os

from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex

if not os.path.exists("data/paul_graham_essay.txt"):
    os.system(
        "wget https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt -P data/"
    )

documents = SimpleDirectoryReader("data").load_data()
index = VectorStoreIndex.from_documents(documents)

query_engine = index.as_query_engine()

# This test does not run correctly if async is used, i.e. not using
# `sync` to convert to sync.

In [ ]:
from trulens.core import Feedback
from trulens.core.app import App
from trulens.providers.huggingface.provider import Dummy

f = (
    Feedback(Dummy().language_match)
    .on_input()
    .on(App.select_context(query_engine))
)

tru_query_engine_recorder = TruLlama(query_engine, feedbacks=[f])

In [ ]:
llm_response, record = tru_query_engine_recorder.with_record(
    query_engine.query, "What did the author do growing up?"
)
record

In [ ]:
run_dashboard(tru, _dev=base, force=True)

In [ ]:
res = record_async.feedback_results[0].result()

In [ ]:
res.result

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine)
# with tru_query_engine_recorder as recording:
llm_response_async, record = await tru_query_engine_recorder.awith_record(
    query_engine.aquery, "What did the author do growing up?"
)

# record_async = recording.get()

In [ ]:
tru_query_engine_recorder = TruLlama(query_engine)
with tru_query_engine_recorder as recording:
    llm_response_async = query_engine.aquery(
        "What did the author do growing up?"
    )

# record_async = recording.get()

In [ ]:
recording.records

In [ ]:
from llama_index.core.base_query_engine import BaseQueryEngine

isinstance(query_engine, BaseQueryEngine)

In [ ]:
query_engine = index.as_query_engine()
tru_query_engine_recorder = TruLlama(query_engine)
with tru_query_engine_recorder as recording:
    llm_response_sync = query_engine.query("What did the author do growing up?")
record_sync = recording.get()